In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision.transforms as transforms

import torch.nn.init as init

In [19]:
torch.manual_seed(1)

In [23]:
import torchvision.datasets as dsets

mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

batch_size = 128

data_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle=True, drop_last=True)

In [22]:
inputs = torch.Tensor(1,1,28,28)
print(inputs.shape)

conv1 = nn.Conv2d(1, 32, 3, padding=1)
pool = nn.MaxPool2d(2)
conv2 = nn.Conv2d(32, 64, 3, padding=1)

out = conv1(inputs)
out = pool(out)
out = conv2(out)
out = pool(out)
print(out.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 64, 7, 7])


In [24]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('current device: ', dev)

torch.manual_seed(7777)
if dev == "cuda:0":
  torch.cuda.manual_seed_all(7777)

current device:  cuda:0


In [25]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [26]:
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

dataloader = DataLoader(
    dataset = mnist_train,
    batch_size = batch_size,  # 각 minibatch의 크기
    shuffle=True,    # Epoch마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꾼다.
    drop_last=True
)

In [27]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(7*7*64, 10, bias=True)
    init.xavier_uniform_(self.fc.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out


In [28]:
model = CNN().to(dev)

In [32]:
criterion = nn.CrossEntropyLoss().to(dev)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(dev)
    Y = Y.to(dev)

    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch:{}] cost={}'.format(epoch+1, avg_cost))
print('Learning Finished!')

  


[Epoch:1] cost=0.06411078572273254
[Epoch:2] cost=0.0458214096724987
[Epoch:3] cost=0.03678667172789574
[Epoch:4] cost=0.031113270670175552
[Epoch:5] cost=0.026076549664139748
[Epoch:6] cost=0.022395260632038116
[Epoch:7] cost=0.01945263147354126
[Epoch:8] cost=0.016586527228355408
[Epoch:9] cost=0.01381582673639059
[Epoch:10] cost=0.012931263074278831
[Epoch:11] cost=0.010135088115930557
[Epoch:12] cost=0.008985794149339199
[Epoch:13] cost=0.007871612906455994
[Epoch:14] cost=0.007842913269996643
[Epoch:15] cost=0.00502536166459322
Learning Finished!


In [35]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(dev)
  Y_test = mnist_test.test_labels.to(dev)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy', accuracy.item())

Accuracy 0.98089998960495


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
